# Лабораторная работа №6. Применение сверточных нейронных сетей (многоклассовая классификация)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

## Задание 1.
Загрузите данные. Разделите исходный набор данных на обучающую и валидационную выборки.

In [17]:
import os
import pandas as pd

dataset_path = os.path.join('data', 'sign-language-mnist')

train_data = pd.read_csv(os.path.join(dataset_path, 'sign_mnist_train.csv'))
test_data = pd.read_csv(os.path.join(dataset_path, 'sign_mnist_test.csv'))

In [18]:
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [4]:
from sklearn.model_selection import train_test_split

y_train = train_data['label'].values
train_data.drop(columns='label', inplace=True)
X_train = train_data.values

y_test = test_data['label'].values
test_data.drop(columns='label', inplace=True)
X_test = test_data.values

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    stratify=y_train
)

X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

## Задание 2.
Реализуйте глубокую нейронную сеть со сверточными слоями. Какое качество классификации получено? Какая архитектура сети была использована?

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, BatchNormalization, Dropout, Flatten

network = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(24, activation='softmax')
])

network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_4 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 12, 12, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)         

In [62]:
# to_categorial из keras работает не совсем так как нужно, у нас классы от 0 до 25, но нет 9-го класса.
# И он возвращает матрицу из 25 столбцов(в 9 столбце у всех нули), а должно быть по логике вещей 24.

def to_categorial_v2(y):
    y_set = set(y)
    y_uniq_ordered_list = sorted(list(y_set))
    num_classes = len(y_set)
    classes_mtx = np.zeros((len(y), num_classes))
    for i, label in enumerate(y):
        j = y_uniq_ordered_list.index(label)
        classes_mtx[i][j] = 1

    return classes_mtx

In [63]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_data_generator(X, y):
    gen = ImageDataGenerator(rescale=1/255.)
    return gen.flow(
        X,
        to_categorial_v2(y),
        batch_size=64,
        seed=42,
        shuffle=True,
    )

train_gen = get_data_generator(X_train, y_train)
val_gen = get_data_generator(X_val, y_val)
test_gen = get_data_generator(X_test, y_test)

In [64]:
to_categorial_v2(y_train).shape

(21964, 24)

In [65]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [66]:
network.fit(
    train_gen, 
    epochs=20,
    validation_data=val_gen,
    workers=4,
)

Train for 344 steps, validate for 86 steps
Epoch 1/20
344/344 [==============================] - 3s 8ms/step - loss: 1.4013 - accuracy: 0.5888 - val_loss: 1.9005 - val_accuracy: 0.3566
Epoch 2/20
344/344 [==============================] - 2s 6ms/step - loss: 0.2979 - accuracy: 0.9017 - val_loss: 0.0507 - val_accuracy: 0.9945
Epoch 3/20
344/344 [==============================] - 2s 6ms/step - loss: 0.1198 - accuracy: 0.9654 - val_loss: 0.0064 - val_accuracy: 0.9995
Epoch 4/20
344/344 [==============================] - 2s 5ms/step - loss: 0.0614 - accuracy: 0.9835 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 5/20
344/344 [==============================] - 2s 6ms/step - loss: 0.0393 - accuracy: 0.9900 - val_loss: 6.3062e-04 - val_accuracy: 1.0000
Epoch 6/20
344/344 [==============================] - 2s 6ms/step - loss: 0.0315 - accuracy: 0.9914 - val_loss: 0.0012 - val_accuracy: 0.9998
Epoch 7/20
344/344 [==============================] - 2s 5ms/step - loss: 0.0200 - accuracy: 0.9943 -

In [67]:
network.evaluate(test_gen)

113/113 [==============================] - 0s 4ms/step - loss: 0.0664 - accuracy: 0.9820


[0.06638175332135793, 0.9820134]

## Задание 3.
Примените дополнение данных (data augmentation). Как это повлияло на качество классификатора?

In [75]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def get_augmented_data_generator(X, y):
    gen = ImageDataGenerator(
        rotation_range=0.15,
        rescale=1./255,
        zoom_range=0.05,
        horizontal_flip=True,
        width_shift_range=0.05,
        height_shift_range=0.05
    )
    
    return gen.flow(
        X,
        to_categorial_v2(y),
        batch_size=64,
        seed=42,
        shuffle=True,
    )


train_gen = get_augmented_data_generator(X_train, y_train)
val_gen = get_augmented_data_generator(X_val, y_val)

In [76]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, BatchNormalization, Dropout, Flatten

network = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(24, activation='softmax')
])

network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
network.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_16 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
batch_normalization_17 (Batc (None, 12, 12, 64)        256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 6, 64)         

In [77]:
network.fit(
    train_gen, 
    epochs=50,
    validation_data=val_gen,
    workers=4,
)

Train for 344 steps, validate for 86 steps
Epoch 1/50
344/344 [==============================] - 6s 17ms/step - loss: 2.1027 - accuracy: 0.3920 - val_loss: 2.7312 - val_accuracy: 0.2227
Epoch 2/50
344/344 [==============================] - 5s 15ms/step - loss: 0.8505 - accuracy: 0.7076 - val_loss: 0.3802 - val_accuracy: 0.9029
Epoch 3/50
344/344 [==============================] - 5s 15ms/step - loss: 0.4969 - accuracy: 0.8307 - val_loss: 0.2604 - val_accuracy: 0.9210
Epoch 4/50
344/344 [==============================] - 5s 15ms/step - loss: 0.3262 - accuracy: 0.8897 - val_loss: 0.1058 - val_accuracy: 0.9729
Epoch 5/50
344/344 [==============================] - 5s 15ms/step - loss: 0.2367 - accuracy: 0.9199 - val_loss: 0.0947 - val_accuracy: 0.9754
Epoch 6/50
344/344 [==============================] - 5s 15ms/step - loss: 0.1870 - accuracy: 0.9351 - val_loss: 0.4100 - val_accuracy: 0.8661
Epoch 7/50
344/344 [==============================] - 5s 15ms/step - loss: 0.1507 - accuracy: 0.949

In [74]:
network.evaluate(test_gen)

113/113 [==============================] - 0s 3ms/step - loss: 0.6653 - accuracy: 0.8773


[0.6653488172894031, 0.8773006]

## Задание 4.
Поэкспериментируйте с готовыми нейронными сетями (например, AlexNet, VGG16, Inception и т.п.), применив передаточное обучение. Как это повлияло на качество классификатора? Можно ли было обойтись без него?
Какой максимальный результат удалось получить на контрольной выборке?

In [87]:
def adjust_data_for_vgg19(data):
    # 3 канала и расширяем до 32 размер картинки
    return np.pad(
        np.concatenate([data, data, data], axis=-1),
        ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant'
    )

X_train_adjusted = adjust_data_for_vgg19(X_train)
X_val_adjusted = adjust_data_for_vgg19(X_val)
X_test_adjusted = adjust_data_for_vgg19(X_test)

In [88]:
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

pre_trained_network = VGG19(input_shape=(32, 32, 3), include_top=False, weights="imagenet")
    
for layer in pre_trained_network.layers[:15]:
    layer.trainable = False

for layer in pre_trained_network.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_network.get_layer('block5_pool')
last_output = last_layer.output
    
x = GlobalAveragePooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(24, activation='softmax')(x)

network = Model(pre_trained_network.input, x)

network.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

network.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [89]:
train_gen = get_data_generator(X_train_adjusted, y_train)
val_gen = get_data_generator(X_val_adjusted, y_val)
test_gen = get_data_generator(X_test_adjusted, y_test)

In [90]:
network.fit(
    train_gen, 
    epochs=20,
    validation_data=val_gen,
    workers=4,
)

Train for 344 steps, validate for 86 steps
Epoch 1/20
344/344 [==============================] - 12s 35ms/step - loss: 2.7378 - accuracy: 0.2033 - val_loss: 1.7463 - val_accuracy: 0.5336
Epoch 2/20
344/344 [==============================] - 12s 34ms/step - loss: 1.4406 - accuracy: 0.5355 - val_loss: 0.7759 - val_accuracy: 0.7849
Epoch 3/20
344/344 [==============================] - 12s 34ms/step - loss: 0.8423 - accuracy: 0.7164 - val_loss: 0.4452 - val_accuracy: 0.8758
Epoch 4/20
344/344 [==============================] - 12s 34ms/step - loss: 0.5513 - accuracy: 0.8139 - val_loss: 0.2971 - val_accuracy: 0.9168
Epoch 5/20
344/344 [==============================] - 12s 34ms/step - loss: 0.3935 - accuracy: 0.8716 - val_loss: 0.1887 - val_accuracy: 0.9554
Epoch 6/20
344/344 [==============================] - 12s 34ms/step - loss: 0.3024 - accuracy: 0.9011 - val_loss: 0.1432 - val_accuracy: 0.9685
Epoch 7/20
344/344 [==============================] - 12s 34ms/step - loss: 0.2240 - accuracy

In [92]:
network.evaluate(test_gen)

113/113 [==============================] - 2s 20ms/step - loss: 0.0563 - accuracy: 0.9822


[0.05627798751425163, 0.9821528]